In [10]:
import pandas as pd
import glob
import numpy as np
import os 
import sys


In [11]:
sheets = np.array(glob.glob( "S*.xlsx"))
print(sheets)

['Summary_Properties_mCPBA.xlsx']


In [12]:
#Remove min, max, stdv, range columns that don't serve ML purposes 
phrases = [   "Unnamed" , "SASA" , "5.0" , "range" , "volume" , "pyramidalization" , "4.0" ]
for i in range (len(sheets)):
    df = pd.read_excel(sheets[i])
    elimCol = [col for col in df.columns if any(phrase in col for phrase in phrases)]
    df_filtered = df.drop(columns=elimCol)
    allColumns = list(df_filtered.columns)
    sterCol = [col for col in allColumns if "Sterimol" in col and "Buried" not in col]

    dfNew = df_filtered.drop(columns = sterCol)
    saveStr = "Properties_MAST_" + str(i) + ".xlsx"

    dfNew.to_excel(saveStr, index=False)

In [13]:
sheets2 = np.array(glob.glob( "P*.xlsx"))


In [14]:
allLabels = []
for i in range (len(sheets2)):
    print(i)
    df = pd.read_excel(sheets2[i])
    print(df.shape)
    column_labels = df.columns.values
    allLabels.append(column_labels)
#Take all column headers for the new Summary_Properties_MAST into an array allLabels

0
(243, 176)


In [7]:
#Remove min, max, stdv, range columns that don't serve ML purposes 
phrases = [   "Unnamed"  ]
for i in range (len(sheets)):
    df = pd.read_excel(sheets[i])
    columns_to_remove = [col for col in df.columns if any(phrase in col for phrase in phrases)]
    df_filtered = df.drop(columns=columns_to_remove)

    saveStr = "Properties_MAST_" + str(i) + ".xlsx"

    df_filtered.to_excel(saveStr, index=False)

In [15]:
common = set(allLabels[0])
for array in allLabels[1:]:
    common.intersection_update(array)
commonLabelsMAST = list(common)
#Find intersections using python's integrated set (works through hashing and iteration)

In [16]:
print(np.sort(commonLabelsMAST))

['%Vbur_C1_2.0Å_Boltz' '%Vbur_C1_2.0Å_Boltz_stdev' '%Vbur_C1_2.0Å_low_E'
 '%Vbur_C1_2.0Å_max' '%Vbur_C1_2.0Å_min' '%Vbur_C1_2.5Å_Boltz'
 '%Vbur_C1_2.5Å_Boltz_stdev' '%Vbur_C1_2.5Å_low_E' '%Vbur_C1_2.5Å_max'
 '%Vbur_C1_2.5Å_min' '%Vbur_C1_3.0Å_Boltz' '%Vbur_C1_3.0Å_Boltz_stdev'
 '%Vbur_C1_3.0Å_low_E' '%Vbur_C1_3.0Å_max' '%Vbur_C1_3.0Å_min'
 '%Vbur_C1_3.5Å_Boltz' '%Vbur_C1_3.5Å_Boltz_stdev' '%Vbur_C1_3.5Å_low_E'
 '%Vbur_C1_3.5Å_max' '%Vbur_C1_3.5Å_min' '%Vbur_C2_2.0Å_Boltz'
 '%Vbur_C2_2.0Å_Boltz_stdev' '%Vbur_C2_2.0Å_low_E' '%Vbur_C2_2.0Å_max'
 '%Vbur_C2_2.0Å_min' '%Vbur_C2_2.5Å_Boltz' '%Vbur_C2_2.5Å_Boltz_stdev'
 '%Vbur_C2_2.5Å_low_E' '%Vbur_C2_2.5Å_max' '%Vbur_C2_2.5Å_min'
 '%Vbur_C2_3.0Å_Boltz' '%Vbur_C2_3.0Å_Boltz_stdev' '%Vbur_C2_3.0Å_low_E'
 '%Vbur_C2_3.0Å_max' '%Vbur_C2_3.0Å_min' '%Vbur_C2_3.5Å_Boltz'
 '%Vbur_C2_3.5Å_Boltz_stdev' '%Vbur_C2_3.5Å_low_E' '%Vbur_C2_3.5Å_max'
 '%Vbur_C2_3.5Å_min' 'Buried_Sterimol_B1_C1_C2_3.0(Å)_Boltz'
 'Buried_Sterimol_B1_C1_C2_3.0(Å)_Boltz_stdev'
 'B

In [17]:
for i in range (len(sheets2)):
    print(sheets2[i])
    if i == 0:
        df = pd.read_excel(sheets2[i])
        #print(df)
        df_MAST = df[commonLabelsMAST]
        #print(df_MAST)
    else:
        df = pd.read_excel(sheets2[i])
        rel_df = df[commonLabelsMAST]
        
        df_MAST = pd.concat([df_MAST, rel_df], axis=0)
    print(df.shape)

    print(df_MAST)
#Take from the new Summary_Properties_MAST, only the data present in the columns of interest and concatenate 

Properties_MAST_0.xlsx
(243, 176)
     %Vbur_C2_2.0Å_low_E  %Vbur_C1_2.5Å_low_E  polar_aniso(au)_max  \
0              94.111570            83.661610              90.2445   
1              95.118802            83.606264             125.2870   
2              91.926007            79.723922             138.9810   
3              94.883135            82.644225             102.6330   
4              93.369060            77.212202             185.2290   
..                   ...                  ...                  ...   
238            86.014979            91.836502             119.4340   
239            94.169680            97.014585             106.7360   
240            91.587035            91.410014             104.7220   
241            97.339876            91.258627             105.8080   
242                  NaN                  NaN              59.5599   

     %Vbur_C2_2.5Å_min   μ_Boltz  %Vbur_C2_3.0Å_Boltz_stdev  \
0            83.938338 -0.144469                   0.000000   

In [18]:
df_sorted = df_MAST[sorted(df_MAST.columns)]

print(df_sorted)
#Revert to Smiles Structure 
df_sorted = df_sorted[["Compound_Name"] + [col for col in df_sorted.columns if col != "Compound_Name"]]

df_sorted.to_excel("sudipALKS_SummaryProps.xlsx", index=False)

     %Vbur_C1_2.0Å_Boltz  %Vbur_C1_2.0Å_Boltz_stdev  %Vbur_C1_2.0Å_low_E  \
0              93.894425                   0.000000            93.898502   
1              94.414566                   0.069379            94.479597   
2              93.032208                   0.467877            92.594267   
3              93.617388                   0.456422            93.840393   
4              92.278787                   0.076893            92.336002   
..                   ...                        ...                  ...   
238            97.964694                   0.160240            97.995222   
239            99.555289                   0.114899            99.586777   
240            97.481921                   0.000000            97.481921   
241            96.654996                   0.049013            96.700671   
242             0.243808                        NaN                  NaN   

     %Vbur_C1_2.0Å_max  %Vbur_C1_2.0Å_min  %Vbur_C1_2.5Å_Boltz  \
0            93.89850

In [29]:
#Load MASTER File 
df_MASTER = pd.read_excel("mCPBAALKS_SummaryProps.xlsx")

smilesDir = "/Users/danielruiz/Downloads/mCPBA_Lit_Database_01_27_25/MasterDataFramemCPBA.csv"

nameDir = "/Users/danielruiz/Downloads/mCPBA_Lit_Database_01_27_25/features/DFTInput/masterDF_KMeansClustered_mCPBA_rdkitmorganDFTInput.csv"


In [30]:
def convertCanonical(str):
    from rdkit import Chem
    print(str)
    mol = Chem.MolFromSmiles(str)
    canonical = Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
    return canonical

In [31]:

nameDF = pd.read_csv(nameDir)
smilesDF = pd.read_csv(smilesDir)
smilesDF.insert(5, "Canonical", 0)
smiles_ =  list(smilesDF["SMILES"])
for i , smile in enumerate(smiles_):
    canonical = convertCanonical(smile)
    smilesDF["Canonical"][i] = canonical


CO[C@@H]1[C@@H](OC)[C@H](c2ccccc2)O[C@@H]2COC(=O)CC/C=C/CCC(=O)O[C@@H]12
C=CC[C@@H](O)[C@@H]1CCCN1C(=O)OCc1ccccc1
OC[C@H]1C=C[C@H](OCc2ccccc2)[C@@H](OCc2ccccc2)[C@@H]1O
COC(=O)/C=C/[C@H](Cc1ccccc1)NC(=O)OC(C)(C)C
COC(=O)/C=C/[C@H](CC(C)C)NC(=O)OC(C)(C)C
C=C(CO)C(O[Si](C)(C)C(C)(C)C)c1ccccc1
C=CCC(O)c1ccc(OCc2ccccc2)c(OC)c1
CCCC/C=C1\CCCC1O
CCCC/C=C1\CCCC12OCCO2
O=C1CC=C[C@@H](c2ccccc2)N1Cc1ccccc1
CC(C)(C)[Si](C)(C)OC[C@H]1O[C@@H](c2ccccc2)C=C[C@@H]1O
CCOC(=O)[C@@H]1CC=CC[C@H]1NC(=O)OCc1ccccc1
CCOC(=O)[C@H]1CC=CC[C@@H]1NC(=O)OCc1ccccc1
C[C@@H]1C[C@@H]2[C@H]([C@@H]3[C@@H]4O[C@@H]4C[C@H]4CCC=C4[C@]31O)C2(C)C
CC/C=C\CCOC(=O)CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC[C@@H]4C[C@H](O)CC[C@]4(C)[C@H]3CC[C@]12C
CC/C=C/CC(=O)O[C@H]1CC[C@@]2(C)[C@@H](CC[C@@H]3[C@@H]2CC[C@]2(C)[C@@H]([C@H](C)CCCC(C)C)CC[C@@H]32)C1
CCC/C=C\CCCCC(=O)OC1CC[C@@]2(C)[C@@H](CC[C@@H]3[C@@H]2CC[C@]2(C)[C@@H]([C@H](C)CCCC(C)C)CC[C@@H]32)C1
OC[C@H]1O[C@H](c2ccccc2)C=C[C@@H]1O
CO[C@@H]1C[C@H]2[C@@H]3CC=C([C@H](C)COC(C)=O)[C@@]3(C)CC

/var/folders/dx/yh8nqkbd21g4hp_8xcg7ts8c0000gn/T/ipykernel_27882/1127663936.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  smilesDF["Canonical"][i] = canonical
/var/folders/dx/yh8nqkbd21g4hp_8xcg7ts8c0000gn/T/ipykernel_27882/1127663936.

CC(=O)N[C@@H]1C=C(COC(C)=O)[C@H](OC(C)=O)[C@@H](OC(C)=O)[C@@H]1OC(C)=O
COC(=O)C1C(=O)C[C@H]2[C@@](C)(CC=C3[C@H](C)C4(CC[C@@]32C)OCCO4)[C@H]1CC(=O)OC(C)(C)C
COC(=O)C1C(=O)C[C@H]2[C@@](C)(CC=C3[C@H](C)C4(CC[C@@]32C)OCCO4)[C@H]1CC(=O)OC(C)(C)C
COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@]1(C=C3)OC(C)(C)OC1(C)OC
CCCCCCC1(CCCCCC/C=C\C[C@@H]2OC(C)(C)O[C@H]2[C@H](COC(c2ccccc2)(c2ccccc2)c2ccccc2)O[Si](C)(C)C(C)(C)C)OCC(C)(C)CO1
C1=C[C@@H]2C[C@H]1CC21CC1
CC1(O)C=CCC1
C=CCOCOCCOC
OC[C@@H]1CC=C[C@H]1O
CC/C=C\CCO[Si](C)(C)C(C)(C)C
CC/C=C/CCO[Si](C)(C)C(C)(C)C
C=CCC(C)(O)CC(=O)OC(C)(C)C
O=C1OC[C@@H]2C=CC[C@H]12
C1=CC2C1[C@H]1O[C@H]1[C@H]1O[C@@H]21
CC(=O)O[C@@H]1C[C@H]2C=C[C@@H]1C2
C=CC(O)CCCCCCC(=O)OC
C/C=C/CCCSc1ccccc1
C=C[C@@H](CO)NC(=O)OC(C)(C)C
C1=C[C@H]2C[C@@H]1[C@]13O[C@H]1C[C@H]1O[C@]213
C1=C[C@H]2C[C@@H]1[C@@]13O[C@@H]1C[C@@H]1O[C@]123
C=CCC1CC(=O)N1[Si](C)(C)C(C)(C)C
COC(=O)C1(Cl)C2C=CCCC21
C[C@@H]1C(=O)[C@H](C)[C@]2(C)C=C[C@H]1O2
CC[C@@H]1C(=O)[C@H](CC)[C@@H]2C=C[C@H]1O2
CCCCCCCCCC/C=C/CCOC

In [32]:
scopeNames = nameDF['ID']
scopeSMILES = smilesDF['SMILES']
dfNames = df_MASTER['Compound_Name']
yields = smilesDF['Yield']
#print(df_MASTER)

In [33]:
def insertIntoDataframe(df1 ,  connectingStr ,connectingStr2 ,  df2 , inputStr , indexInterest  ):
    #First create an empty column of zeros at the index of interest and call it inputStr 
    df1.insert(indexInterest, inputStr, 0)
    
    col1 = list(df1[connectingStr])

    col2 = list(df2[connectingStr2])
    insertCol = list(df2[inputStr])
    

    for i , str in enumerate (col1):
        if str in col2:
            index = col2.index(str)
            insertVal = insertCol[index]
            #df1[inputStr][i] = insertVal
            df1.loc[i, inputStr] = insertVal
    return df1

In [34]:
display(df_MASTER)

,Compound_Name,%Vbur_C1_2.0Å_Boltz,%Vbur_C1_2.0Å_Boltz_stdev,%Vbur_C1_2.0Å_low_E,%Vbur_C1_2.0Å_max,%Vbur_C1_2.0Å_min,%Vbur_C1_2.5Å_Boltz,%Vbur_C1_2.5Å_Boltz_stdev,%Vbur_C1_2.5Å_low_E,%Vbur_C1_2.5Å_max,...,μ_Boltz,μ_Boltz_stdev,μ_low_E,μ_max,μ_min,ω_Boltz,ω_Boltz_stdev,ω_low_E,ω_max,ω_min
0,0_0_Alk_000000,93.894425,0.000000,93.898502,93.898502,93.898502,83.657977,0.000000,83.661610,83.661610,...,-0.144469,0.000000,-0.144475,-0.144475,-0.144475,0.033099,0.000000,0.03310,0.03310,0.03310
1,0_0_Alk_000001,94.414566,0.069379,94.479597,94.479597,94.289127,83.668837,0.192309,83.606264,84.167860,...,-0.145732,0.000836,-0.145805,-0.135080,-0.147095,0.039794,0.000578,0.03979,0.04066,0.03246
2,0_0_Alk_000002,93.032208,0.467877,92.594267,93.556302,92.594267,80.115166,0.412976,79.723922,80.588293,...,-0.136949,0.000672,-0.136350,-0.136345,-0.137700,0.033505,0.000765,0.03282,0.03436,0.03282
3,0_0_Alk_000003,93.617388,0.456422,93.840393,95.331870,93.239928,82.632556,0.731044,82.644225,86.085428,...,-0.144674,0.000646,-0.145215,-0.141670,-0.145315,0.035975,0.000400,0.03633,0.03633,0.03451
4,0_0_Alk_000004,92.278787,0.076893,92.336002,92.439308,92.248838,77.127477,0.066764,77.212202,77.252898,...,-0.144120,0.003551,-0.139950,-0.139950,-0.149715,0.034744,0.002383,0.03193,0.03828,0.03193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0_0_Alk_000247,97.964694,0.160240,97.995222,98.017820,96.916968,91.793297,0.205815,91.836502,91.844641,...,-0.126871,0.000348,-0.126710,-0.125620,-0.131140,0.024181,0.000134,0.02410,0.02572,0.02354
239,0_0_Alk_000248,99.555289,0.114899,99.586777,99.586777,98.808755,96.952353,0.330367,97.014585,97.151322,...,-0.136217,0.000357,-0.136260,-0.134445,-0.137065,0.030594,0.000119,0.03062,0.03062,0.02996
240,0_0_Alk_000249,97.481921,0.000000,97.481921,97.481921,97.481921,91.410014,0.000000,91.410014,91.410014,...,-0.117620,0.000000,-0.117620,-0.117620,-0.117620,0.020790,0.000000,0.02079,0.02079,0.02079
241,0_0_Alk_000250,96.654996,0.049013,96.700671,96.858858,96.607051,91.171697,0.111972,91.258627,91.650931,...,-0.120812,0.000889,-0.120950,-0.119230,-0.123320,0.022057,0.000340,0.02201,0.02352,0.02169


In [35]:
dfNew = insertIntoDataframe(df_MASTER ,  "Compound_Name" ,"ID" ,  nameDF , "SMILES" , 0  )

/var/folders/dx/yh8nqkbd21g4hp_8xcg7ts8c0000gn/T/ipykernel_27882/1792382023.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'CO[C@@H]1[C@@H](OC)[C@H](c2ccccc2)O[C@@H]2COC(=O)CC/C=C/CCC(=O)O[C@@H]12' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1.loc[i, inputStr] = insertVal


In [36]:
display(dfNew)


,SMILES,Compound_Name,%Vbur_C1_2.0Å_Boltz,%Vbur_C1_2.0Å_Boltz_stdev,%Vbur_C1_2.0Å_low_E,%Vbur_C1_2.0Å_max,%Vbur_C1_2.0Å_min,%Vbur_C1_2.5Å_Boltz,%Vbur_C1_2.5Å_Boltz_stdev,%Vbur_C1_2.5Å_low_E,...,μ_Boltz,μ_Boltz_stdev,μ_low_E,μ_max,μ_min,ω_Boltz,ω_Boltz_stdev,ω_low_E,ω_max,ω_min
0,CO[C@@H]1[C@@H](OC)[C@H](c2ccccc2)O[C@@H]2COC(...,0_0_Alk_000000,93.894425,0.000000,93.898502,93.898502,93.898502,83.657977,0.000000,83.661610,...,-0.144469,0.000000,-0.144475,-0.144475,-0.144475,0.033099,0.000000,0.03310,0.03310,0.03310
1,COc1cc(OC(C)=O)c(C(=O)C(C)C)c2c1[C@H]1C[C@H](C...,0_0_Alk_000001,94.414566,0.069379,94.479597,94.479597,94.289127,83.668837,0.192309,83.606264,...,-0.145732,0.000836,-0.145805,-0.135080,-0.147095,0.039794,0.000578,0.03979,0.04066,0.03246
2,O=C1[C@H]2[C@H](c3ccccc3)[C@@H]3C=C[C@]2(O3)[C...,0_0_Alk_000002,93.032208,0.467877,92.594267,93.556302,92.594267,80.115166,0.412976,79.723922,...,-0.136949,0.000672,-0.136350,-0.136345,-0.137700,0.033505,0.000765,0.03282,0.03436,0.03282
3,COC(=O)C(C(=O)OC)[C@@H]1C=C[C@@]2(C(=O)OC)Cc3c...,0_0_Alk_000003,93.617388,0.456422,93.840393,95.331870,93.239928,82.632556,0.731044,82.644225,...,-0.144674,0.000646,-0.145215,-0.141670,-0.145315,0.035975,0.000400,0.03633,0.03633,0.03451
4,CC(=O)[C@H]1CC=C[C@H]2[C@@H](OCc3ccccc3)[C@H](...,0_0_Alk_000004,92.278787,0.076893,92.336002,92.439308,92.248838,77.127477,0.066764,77.212202,...,-0.144120,0.003551,-0.139950,-0.139950,-0.149715,0.034744,0.002383,0.03193,0.03828,0.03193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,C=C(C)[C@H]1CC[C@H]2[C@@H]3CC[C@@H]4C[C@@H](OC...,0_0_Alk_000247,97.964694,0.160240,97.995222,98.017820,96.916968,91.793297,0.205815,91.836502,...,-0.126871,0.000348,-0.126710,-0.125620,-0.131140,0.024181,0.000134,0.02410,0.02572,0.02354
239,CC1=CC[C@@H]2[C@@H](CC[C@@]2(C)O)C(C)(C)[C@H]1...,0_0_Alk_000248,99.555289,0.114899,99.586777,99.586777,98.808755,96.952353,0.330367,97.014585,...,-0.136217,0.000357,-0.136260,-0.134445,-0.137065,0.030594,0.000119,0.03062,0.03062,0.02996
240,CC1=CC[C@H]2[C@@H]3[C@H]4OC(C)(C)O[C@@H]4[C@H]...,0_0_Alk_000249,97.481921,0.000000,97.481921,97.481921,97.481921,91.410014,0.000000,91.410014,...,-0.117620,0.000000,-0.117620,-0.117620,-0.117620,0.020790,0.000000,0.02079,0.02079,0.02079
241,C[C@]12CCC3C4=C(CC5(CC4)OCCO5)[C@H](O)CC3C1CCC...,0_0_Alk_000250,96.654996,0.049013,96.700671,96.858858,96.607051,91.171697,0.111972,91.258627,...,-0.120812,0.000889,-0.120950,-0.119230,-0.123320,0.022057,0.000340,0.02201,0.02352,0.02169


In [37]:
dfNew = insertIntoDataframe(dfNew ,  "SMILES" ,"Canonical" ,  smilesDF , "Yield" , 2  )

/var/folders/dx/yh8nqkbd21g4hp_8xcg7ts8c0000gn/T/ipykernel_27882/1792382023.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '23.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1.loc[i, inputStr] = insertVal


In [38]:
display(dfNew)


,SMILES,Compound_Name,Yield,%Vbur_C1_2.0Å_Boltz,%Vbur_C1_2.0Å_Boltz_stdev,%Vbur_C1_2.0Å_low_E,%Vbur_C1_2.0Å_max,%Vbur_C1_2.0Å_min,%Vbur_C1_2.5Å_Boltz,%Vbur_C1_2.5Å_Boltz_stdev,...,μ_Boltz,μ_Boltz_stdev,μ_low_E,μ_max,μ_min,ω_Boltz,ω_Boltz_stdev,ω_low_E,ω_max,ω_min
0,CO[C@@H]1[C@@H](OC)[C@H](c2ccccc2)O[C@@H]2COC(...,0_0_Alk_000000,27.0,93.894425,0.000000,93.898502,93.898502,93.898502,83.657977,0.000000,...,-0.144469,0.000000,-0.144475,-0.144475,-0.144475,0.033099,0.000000,0.03310,0.03310,0.03310
1,COc1cc(OC(C)=O)c(C(=O)C(C)C)c2c1[C@H]1C[C@H](C...,0_0_Alk_000001,42.0,94.414566,0.069379,94.479597,94.479597,94.289127,83.668837,0.192309,...,-0.145732,0.000836,-0.145805,-0.135080,-0.147095,0.039794,0.000578,0.03979,0.04066,0.03246
2,O=C1[C@H]2[C@H](c3ccccc3)[C@@H]3C=C[C@]2(O3)[C...,0_0_Alk_000002,57.0,93.032208,0.467877,92.594267,93.556302,92.594267,80.115166,0.412976,...,-0.136949,0.000672,-0.136350,-0.136345,-0.137700,0.033505,0.000765,0.03282,0.03436,0.03282
3,COC(=O)C(C(=O)OC)[C@@H]1C=C[C@@]2(C(=O)OC)Cc3c...,0_0_Alk_000003,72.0,93.617388,0.456422,93.840393,95.331870,93.239928,82.632556,0.731044,...,-0.144674,0.000646,-0.145215,-0.141670,-0.145315,0.035975,0.000400,0.03633,0.03633,0.03451
4,CC(=O)[C@H]1CC=C[C@H]2[C@@H](OCc3ccccc3)[C@H](...,0_0_Alk_000004,84.0,92.278787,0.076893,92.336002,92.439308,92.248838,77.127477,0.066764,...,-0.144120,0.003551,-0.139950,-0.139950,-0.149715,0.034744,0.002383,0.03193,0.03828,0.03193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,C=C(C)[C@H]1CC[C@H]2[C@@H]3CC[C@@H]4C[C@@H](OC...,0_0_Alk_000247,86.0,97.964694,0.160240,97.995222,98.017820,96.916968,91.793297,0.205815,...,-0.126871,0.000348,-0.126710,-0.125620,-0.131140,0.024181,0.000134,0.02410,0.02572,0.02354
239,CC1=CC[C@@H]2[C@@H](CC[C@@]2(C)O)C(C)(C)[C@H]1...,0_0_Alk_000248,32.0,99.555289,0.114899,99.586777,99.586777,98.808755,96.952353,0.330367,...,-0.136217,0.000357,-0.136260,-0.134445,-0.137065,0.030594,0.000119,0.03062,0.03062,0.02996
240,CC1=CC[C@H]2[C@@H]3[C@H]4OC(C)(C)O[C@@H]4[C@H]...,0_0_Alk_000249,57.0,97.481921,0.000000,97.481921,97.481921,97.481921,91.410014,0.000000,...,-0.117620,0.000000,-0.117620,-0.117620,-0.117620,0.020790,0.000000,0.02079,0.02079,0.02079
241,C[C@]12CCC3C4=C(CC5(CC4)OCCO5)[C@H](O)CC3C1CCC...,0_0_Alk_000250,63.1,96.654996,0.049013,96.700671,96.858858,96.607051,91.171697,0.111972,...,-0.120812,0.000889,-0.120950,-0.119230,-0.123320,0.022057,0.000340,0.02201,0.02352,0.02169


In [39]:
import json
alkenePool = "/Users/danielruiz/Downloads/Alkenes/alkenePool/substratePool.json"
with open(alkenePool, 'r') as f:
    alkeneDict = json.load(f)
canonicalSMILES = list(alkeneDict.keys())

In [40]:
def convertCanonical(str):
    from rdkit import Chem
    mol = Chem.MolFromSmiles(str)
    canonical = Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
    return canonical

In [41]:
smiles1 = list(dfNew["SMILES"])

dfNew.insert(3, "ChemistryType", 0)
for i , smile in enumerate(smiles1):
    canonical = convertCanonical(smile)
    if canonical in canonicalSMILES:
        chemType = int(alkeneDict[canonical])
        dfNew["ChemistryType"][i] = chemType
        
    


/var/folders/dx/yh8nqkbd21g4hp_8xcg7ts8c0000gn/T/ipykernel_27882/3805341168.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dfNew["ChemistryType"][i] = chemType
/var/folders/dx/yh8nqkbd21g4hp_8xcg7ts8c0000gn/T/ipykernel_27882/3805341168.

In [42]:
dfNew.to_csv("mCPBAALKS_MASTDF.csv", index=False)

In [43]:
display(dfNew)


,SMILES,Compound_Name,Yield,ChemistryType,%Vbur_C1_2.0Å_Boltz,%Vbur_C1_2.0Å_Boltz_stdev,%Vbur_C1_2.0Å_low_E,%Vbur_C1_2.0Å_max,%Vbur_C1_2.0Å_min,%Vbur_C1_2.5Å_Boltz,...,μ_Boltz,μ_Boltz_stdev,μ_low_E,μ_max,μ_min,ω_Boltz,ω_Boltz_stdev,ω_low_E,ω_max,ω_min
0,CO[C@@H]1[C@@H](OC)[C@H](c2ccccc2)O[C@@H]2COC(...,0_0_Alk_000000,27.0,1,93.894425,0.000000,93.898502,93.898502,93.898502,83.657977,...,-0.144469,0.000000,-0.144475,-0.144475,-0.144475,0.033099,0.000000,0.03310,0.03310,0.03310
1,COc1cc(OC(C)=O)c(C(=O)C(C)C)c2c1[C@H]1C[C@H](C...,0_0_Alk_000001,42.0,1,94.414566,0.069379,94.479597,94.479597,94.289127,83.668837,...,-0.145732,0.000836,-0.145805,-0.135080,-0.147095,0.039794,0.000578,0.03979,0.04066,0.03246
2,O=C1[C@H]2[C@H](c3ccccc3)[C@@H]3C=C[C@]2(O3)[C...,0_0_Alk_000002,57.0,1,93.032208,0.467877,92.594267,93.556302,92.594267,80.115166,...,-0.136949,0.000672,-0.136350,-0.136345,-0.137700,0.033505,0.000765,0.03282,0.03436,0.03282
3,COC(=O)C(C(=O)OC)[C@@H]1C=C[C@@]2(C(=O)OC)Cc3c...,0_0_Alk_000003,72.0,1,93.617388,0.456422,93.840393,95.331870,93.239928,82.632556,...,-0.144674,0.000646,-0.145215,-0.141670,-0.145315,0.035975,0.000400,0.03633,0.03633,0.03451
4,CC(=O)[C@H]1CC=C[C@H]2[C@@H](OCc3ccccc3)[C@H](...,0_0_Alk_000004,84.0,1,92.278787,0.076893,92.336002,92.439308,92.248838,77.127477,...,-0.144120,0.003551,-0.139950,-0.139950,-0.149715,0.034744,0.002383,0.03193,0.03828,0.03193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,C=C(C)[C@H]1CC[C@H]2[C@@H]3CC[C@@H]4C[C@@H](OC...,0_0_Alk_000247,86.0,1,97.964694,0.160240,97.995222,98.017820,96.916968,91.793297,...,-0.126871,0.000348,-0.126710,-0.125620,-0.131140,0.024181,0.000134,0.02410,0.02572,0.02354
239,CC1=CC[C@@H]2[C@@H](CC[C@@]2(C)O)C(C)(C)[C@H]1...,0_0_Alk_000248,32.0,1,99.555289,0.114899,99.586777,99.586777,98.808755,96.952353,...,-0.136217,0.000357,-0.136260,-0.134445,-0.137065,0.030594,0.000119,0.03062,0.03062,0.02996
240,CC1=CC[C@H]2[C@@H]3[C@H]4OC(C)(C)O[C@@H]4[C@H]...,0_0_Alk_000249,57.0,1,97.481921,0.000000,97.481921,97.481921,97.481921,91.410014,...,-0.117620,0.000000,-0.117620,-0.117620,-0.117620,0.020790,0.000000,0.02079,0.02079,0.02079
241,C[C@]12CCC3C4=C(CC5(CC4)OCCO5)[C@H](O)CC3C1CCC...,0_0_Alk_000250,63.1,1,96.654996,0.049013,96.700671,96.858858,96.607051,91.171697,...,-0.120812,0.000889,-0.120950,-0.119230,-0.123320,0.022057,0.000340,0.02201,0.02352,0.02169
